## Install

In [ ]:
!pip install python-dotenv langchain-openai langchain_community azure-core azure-search-documents==11.5.1 azure-storage-blob azure-identity openai aiohttp tiktoken --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.7/297.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.9/187.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curr

## Getting URL maps (skip if you are indexing from scratch)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
with open('/content/drive/MyDrive/statpearls-2025-03-02-mappings (1).json') as json_data:
    d = json.load(json_data)

## Keys

In [ ]:
storage_account_url = "https://stpeterswede219286479994.blob.core.windows.net"
storage_account_key = ""

embedding_endpoint = "https://peter-sweden.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2023-05-15"

AZURE_SEARCH_SERVICE: str = "https://peter-search-sweden.search.windows.net"
AZURE_SEARCH_KEY: str = ""
AZURE_OPENAI_ACCOUNT: str = "https://peter-sweden.openai.azure.com/"
AZURE_OPENAI_KEY: str = ""
AZURE_AI_MULTISERVICE_ACCOUNT: str = "https://sweden-multi-service.cognitiveservices.azure.com/"
AZURE_AI_MULTISERVICE_KEY: str = ""
AZURE_STORAGE_CONNECTION: str = ""

from azure.core.credentials import AzureKeyCredential
AZURE_SEARCH_CREDENTIAL = AzureKeyCredential(AZURE_SEARCH_KEY)

## Extracting data and reference map

In [ ]:
import requests

url = 'https://ftp.ncbi.nlm.nih.gov/pub/litarch/3d/12/statpearls_NBK430685.tar.gz'
target_path = 'statpearls-2025-03-02.tar.gz'

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())


In [ ]:
# importing the "tarfile" module
import tarfile

# open file
file = tarfile.open(target_path)

# extracting file
file.extractall('./statpearls')

file.close()

In [ ]:
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm
import glob

filepaths = []
for root, dirs, files in os.walk("./statpearls"):
  for file in files:
    if file.endswith(".nxml"):
      filepaths.append(os.path.join(root, file))

filepaths.remove('./statpearls/statpearls_NBK430685/TOC.nxml') #contents
filepaths.remove('./statpearls/statpearls_NBK430685/contributors.nxml') #contributors

mappings = {}

# Assuming the first .nxml file is desired
for filename in tqdm(filepaths):
  #print(filename)
  tree = ET.parse(filename)
  notags = ET.tostring(tree.getroot(), encoding='utf-8',method='text').decode("utf-8")

  replace_str = '\n  \n    statpearls\n    \n      StatPearls\n    \n    \n      01\n      2025\n    \n    \n      StatPearls Publishing\n      Treasure Island (FL)\n    \n    \n      Copyright © 2025, StatPearls Publishing LLC\n      \n        \nThis book is distributed under the terms of the Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)\n(\nhttp://creativecommons.org/licenses/by-nc-nd/4.0/\n), which permits others to distribute the work, provided that the article is not altered or used commercially. You are not required to obtain permission to distribute this article, provided that you credit the author and journal.\n\n      \n    \n    \n      \n        books-source-type\n        Database\n      \n    \n    \n      The intent of StatPearls review books and articles is to identify knowledge deficits and assist you in the learning process. Review books and articles are not intended to be a source of the knowledge base of medicine. The authors and editors do not warrant the information is complete or accurate. The reader is encouraged to verify content and questions in several references. All drug indications and dosages should be verified before administration.\n      The authors and editors would like to extend special thanks to Erin Hughes and Gerson Rubio for their editorial support.\n    \n  \n  \n    \n      \n        '
  replace_st1 = '\n  \n    statpearls\n    \n      StatPearls\n    \n    \n      06\n      2017\n    \n    \n      StatPearls Publishing\n      Treasure Island (FL)\n    \n    \n      Copyright © 2017, StatPearls Publishing LLC\n      \n        \nThis book is distributed under the terms of the Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)\n(\nhttp://creativecommons.org/licenses/by-nc-nd/4.0/\n), which permits others to distribute the work, provided that the article is not altered or used commercially. You are not required to obtain permission to distribute this article, provided that you credit the author and journal.\n\n      \n    \n    \n      \n        books-source-type\n        Database\n      \n    \n    \n      The intent of StatPearls review books and articles is to identify knowledge deficits and assist you in the learning process. Review books and articles are not intended to be a source of the knowledge base of medicine. The authors and editors do not warrant the information is complete or accurate. The reader is encouraged to verify content and questions in several references. All drug indications and dosages should be verified before administration.\n      The authors and editors would like to extend special thanks to Erin Hughes and Gerson Rubio for their editorial support.\n    \n  \n  \n    \n      \n        '
  replace_st2 = '\n  \n    statpearls\n    \n      StatPearls\n    \n    \n      01\n      2025\n    \n    \n      StatPearls Publishing\n      Treasure Island (FL)\n    \n    \n      Copyright © 2025, StatPearls Publishing LLC\n      \n         This book is distributed under the terms of the Creative Commons\nAttribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0) (\nhttp://creativecommons.org/licenses/by-nc-nd/4.0/ ), which permits others to\ndistribute the work, provided that the article is not altered or used commercially. You\nare not required to obtain permission to distribute this article, provided that you credit\nthe author and journal. \n      \n    \n    \n      \n        books-source-type\n        Database\n      \n    \n    \n      The intent of StatPearls review books and articles is to identify knowledge deficits and\nassist you in the learning process. Review books and articles are not intended to be a\nsource of the knowledge base of medicine. The authors and editors do not warrant the\ninformation is complete or accurate. The reader is encouraged to verify content and\nquestions in several references. All drug indications and dosages should be verified before\nadministration.\n      The authors and editors would like to extend special thanks to Erin Hughes and Gerson Rubio\nfor their editorial support.\n    \n  \n  \n    \n      \n        '
  replace_st3 = '\n  \n    statpearls\n    \n      StatPearls\n    \n    \n      06\n      2017\n    \n    \n      StatPearls Publishing\n      Treasure Island (FL)\n    \n    \n      Copyright © 2017, StatPearls Publishing LLC\n      \n        This book is distributed under the terms of the Creative Commons Attribution 4.0 International License (http://creativecommons.org/licenses/by/4.0/), which permits use, duplication, adaptation, distribution, and reproduction in any medium or format, as long as you give appropriate credit to the original author(s) and the source, a link is provided to the Creative Commons license, and any changes made are indicated.\n      \n    \n    \n      \n        books-source-type\n        Database\n      \n    \n    \n      The intent of StatPearls review books and articles is to identify knowledge deficits and assist you in the learning process. Review books and articles are not intended to be a source of the knowledge base of medicine. The authors and editors do not warrant the information is complete or accurate. The reader is encouraged to verify content and questions in several references. All drug indications and dosages should be verified before administration.\n      The authors and editors would like to extend special thanks to Richard Miller, Tiffany Sneden, Erin Hughes, Beata Beatty, and Gerson Rubio for their editorial support and Susan Oliver for editorial team management.\n    \n  \n  \n    \n      \n        '

  notags = notags.replace(replace_str, '')
  notags = notags.replace(replace_st1, '')
  notags = notags.replace(replace_st2, '')
  notags = notags.replace(replace_st3, '')

  save_name = notags.strip().split('\n')[0]
  save_name = save_name.replace('/', ' ')
  save_name = save_name.replace('*', ' ')

  final = notags.strip().split('References')[0]
  final = final.replace('Review Questions', '')
  final = final.replace('Access free multiple choice questions on this topic.', '')
  final = final.replace('Comment on this article.', '')
  final = final.replace('To access free multiple choice questions on this topic, click here.', '')
  final = final.replace('Click here for a simplified version.', '')

  final2 = "".join([s for s in final.splitlines(True) if s.strip()])
  #print(final2)
  if 'statpearls' in save_name:
    print('statpearls in name')
    print(filename)
    print(save_name)
    print(final2)
    break
  if 'This book is' in save_name:
    print('This book is in name')
    print(filename)
  #print(save_name)
  try:
    number = filename.split('article-')[1].split('.')[0]
  except IndexError:
    print('URL mapping Error')
    print(filename)
  mappings[save_name] = 'https://www.statpearls.com/point-of-care/' + str(number)
  with open('./statpearls/' + save_name + '.txt','w+', encoding="utf-8") as f:
      f.write(final2)


100%|██████████| 9603/9603 [00:43<00:00, 219.59it/s]


In [ ]:
for key in mappings.keys():
  if 'near' in key.lower():
    print(key)

Linear IGA Dermatosis
Drowning, Near


In [ ]:
for key in mappings.keys():
  if 'case study' in key.lower():
    print(key)
    print(mappings[key])

Case Study: 32-Year-Old Male Presenting with Right Lower Quadrant Abdominal Pain (Archived)
https://www.statpearls.com/point-of-care/41441
Case Study: 60-Year-Old Female Presenting With Shortness of Breath (Archived)
https://www.statpearls.com/point-of-care/40870
Case Study: 86-Year-Old Male With Leg Swelling After Hip Surgery (Archived)
https://www.statpearls.com/point-of-care/41589
Case Study: 33-Year-Old Female Presents with Chronic SOB and Cough (Archived)
https://www.statpearls.com/point-of-care/41348
Case Study: 8-Year-Old With a Murmur and Hypertension (Archived)
https://www.statpearls.com/point-of-care/41059
Case Study: 24-Year-Old Male Presenting With Polyarthralgias (Archived)
https://www.statpearls.com/point-of-care/40888
Case Study: 26-Year-Old Male Presenting With New Onset Seizure and History of PTSD (Archived)
https://www.statpearls.com/point-of-care/40892


In [ ]:
import json

with open('statpearls-2025-03-02-mappings.json', 'w') as fp:
    json.dump(mappings, fp)

In [ ]:
mappings['Minimal Change Disease']

'https://www.statpearls.com/point-of-care/25173'

In [ ]:
mappings['Nephrotic Syndrome']

'https://www.statpearls.com/point-of-care/25713'

In [ ]:
mappings['Congenital Nephrotic Syndrome']

'https://www.statpearls.com/point-of-care/132261'

## Uploading to Azure Storage

In [ ]:
textpaths = []
for root, dirs, files in os.walk("./statpearls"):
  for file in files:
    if file.endswith(".txt"):
      textpaths.append(os.path.join(root, file))

In [ ]:
for text in textpaths:
  snippet = text.split('statpearls/')[1]
  snippet = snippet.split('.txt')[0]
  #print(snippet)
  if snippet not in mappings.keys():
    print(snippet)
    textpaths.remove(text)


statpearls
Editorial Board
statpearls_NBK430685/license


In [ ]:
len(textpaths)

9559

In [ ]:
from azure.storage.blob import BlobServiceClient

blob_service_client = BlobServiceClient(storage_account_url, storage_account_key)

container_client = blob_service_client.create_container(name='statpearls-2025-03-02')

In [ ]:
for text in tqdm(textpaths):
  snippet = text.split('statpearls/')[1]
  snippet = snippet.split('.txt')[0]
  blob_client = blob_service_client.get_blob_client(container="statpearls-2025-03-02", blob=snippet)
  with open(text, "rb") as data:
      blob_client.upload_blob(data, overwrite=True)


100%|██████████| 9559/9559 [18:58<00:00,  8.40it/s]


## Indexing

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SearchIndex
)

pre_name = 'statpearls-2025-03-02'

# Create a search index
index_name = pre_name + '-idx'
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE, credential=AZURE_SEARCH_CREDENTIAL)
fields = [
    SearchField(name="parent_id", type=SearchFieldDataType.String),
    SearchField(name="title", type=SearchFieldDataType.String),
    SearchField(name="chunk_id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),
    SearchField(name="text_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=3072, vector_search_profile_name="myHnswProfile")
    ]

# Configure the vector search configuration
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer_name="myOpenAI",
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            vectorizer_name="myOpenAI",
            kind="azureOpenAI",
            parameters=AzureOpenAIVectorizerParameters(
                resource_url=AZURE_OPENAI_ACCOUNT,
                deployment_name="text-embedding-3-large",
                model_name="text-embedding-3-large",
                api_key=AZURE_OPENAI_KEY
            ),
        ),
    ],
)

In [ ]:
# Create the search index
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)
result = index_client.create_or_update_index(index)
print(f"{result.name} created")

statpearls-2025-03-02-idx created


In [ ]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection
)

# Create a data source
indexer_client = SearchIndexerClient(endpoint=AZURE_SEARCH_SERVICE, credential=AZURE_SEARCH_CREDENTIAL)
container = SearchIndexerDataContainer(name=pre_name)
data_source_connection = SearchIndexerDataSourceConnection(
    name = pre_name + '-ds',
    type="azureblob",
    connection_string=AZURE_STORAGE_CONNECTION,
    container=container
)
data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'statpearls-2025-03-02-ds' created or updated


In [ ]:
from azure.search.documents.indexes.models import (
    SplitSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    AzureOpenAIEmbeddingSkill,
    EntityRecognitionSkill,
    SearchIndexerIndexProjection,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    IndexProjectionMode,
    SearchIndexerSkillset,
    CognitiveServicesAccountKey
)

# Create a skillset
skillset_name = pre_name + '-ss'

split_skill = SplitSkill(
    description="Split skill to chunk documents",
    text_split_mode="pages",
    context="/document",
    maximum_page_length=4000,
    page_overlap_length=100,
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/content"),
    ],
    outputs=[
        OutputFieldMappingEntry(name="textItems", target_name="pages")
    ],
)

embedding_skill = AzureOpenAIEmbeddingSkill(
    description="Skill to generate embeddings via Azure OpenAI",
    context="/document/pages/*",
    resource_url=AZURE_OPENAI_ACCOUNT,
    deployment_name="text-embedding-3-large",
    model_name="text-embedding-3-large",
    dimensions=3072,
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/pages/*"),
    ],
    outputs=[
        OutputFieldMappingEntry(name="embedding", target_name="text_vector")
    ],
)

index_projections = SearchIndexerIndexProjection(
    selectors=[
        SearchIndexerIndexProjectionSelector(
            target_index_name=index_name,
            parent_key_field_name="parent_id",
            source_context="/document/pages/*",
            mappings=[
                InputFieldMappingEntry(name="chunk", source="/document/pages/*"),
                InputFieldMappingEntry(name="text_vector", source="/document/pages/*/text_vector"),
                InputFieldMappingEntry(name="title", source="/document/metadata_storage_name"),
            ],
        ),
    ],
    parameters=SearchIndexerIndexProjectionsParameters(
        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS
    ),
)

cognitive_services_account = CognitiveServicesAccountKey(key=AZURE_AI_MULTISERVICE_KEY)

skills = [split_skill, embedding_skill]

skillset = SearchIndexerSkillset(
    name=skillset_name,
    description="Skillset to chunk documents and generating embeddings",
    skills=skills,
    index_projection=index_projections,
    cognitive_services_account=cognitive_services_account
)

client = SearchIndexerClient(endpoint=AZURE_SEARCH_SERVICE, credential=AZURE_SEARCH_CREDENTIAL)
client.create_or_update_skillset(skillset)
print(f"{skillset.name} created")

statpearls-2025-03-02-ss created


In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndexer,
    FieldMapping
)

# Create an indexer
indexer_name = pre_name + '-idxr'

indexer_parameters = None

indexer = SearchIndexer(
    name=indexer_name,
    description="Indexer to index documents and generate embeddings",
    skillset_name=skillset_name,
    target_index_name=index_name,
    data_source_name=data_source.name,
    # Map the metadata_storage_name field to the title field in the index to display the PDF title in the search results
    field_mappings=[FieldMapping(source_field_name="metadata_storage_name", target_field_name="title")],
    parameters=indexer_parameters
)

# Create and run the indexer
indexer_client = SearchIndexerClient(endpoint=AZURE_SEARCH_SERVICE, credential=AZURE_SEARCH_CREDENTIAL)
indexer_result = indexer_client.create_or_update_indexer(indexer)

print(f' {indexer_name} is created and running. Give the indexer a few minutes before running a query.')

 statpearls-2025-03-02-idxr is created and running. Give the indexer a few minutes before running a query.


## Prompts

In [ ]:
UNIVERSAL_SYSTEM_PROMPT = '''You are a helpful medical assistant, an expert specialist in every medical field!
First you'll be given the patient EHR data to pre-process as well as any existing user chat history. Note that the user might later add updates to this patient data, but all your conversation will either be general or revolve around this particular patient.
Then, you'll be given your detailed task. Read your task description carefully before making any decisions. Never deviate from this task description and treat it like your most important system prompt. If you did not get your detailed task description, pause and let the user know.
'''

In [ ]:
STRUCTURED_DX_PROMPT = '''
# SCOPE
Name: DX Tool.
Purpose: Generate differential diagnoses based on clinical information. The diagnoses will then be used to look up relevant sections of clinical guidelines, which will then be used as sources by the management planning tool, which creates a comprehensive treatment plan for the patient.
Operational Limits: Strictly limited to diagnostic analysis. Disregard all other requests in the input (e.g., for MX, coding, clinical documentation) as they are out of scope for this tool and are handled elsewhere.

# ROLE
- You are a specialist in every medical field.
- You know all the latest specialty-specific guidelines, every possible human pathophysiological process, and every disease state described in any medical literature, including:
  - Diseases that are extremely rare.
  - Diseases that are poorly described.
  - Diseases limited to certain regions.
- In addition to your mastery of medicine, you are a distinguished pharmacologist with a deep understanding of drug mechanisms, pharmacokinetics, pharmacodynamics, and potential interactions.
- You pay extreme attention to every detail of a case to determine if it could possibly align with any diagnosis you know of.
- You avoid anchoring, confirmation, availability, premature closure, and framing bias.
- You consider a case from all angles to determine the right diagnosis.
- Because of the aforementioned traits, another clinician is consulting you.
- Your role is to:
  - Suggest probable diagnoses and conditions to medical doctors.
  - Substantiate your suggestions with evidence drawn from the data provided.

# TASK
- Using the patient information from the context you have access to, suggest the correct diagnoses and conditions, followed by the key evidence points supporting each diagnosis.
- Include previous diagnoses and pertinent information about the patient’s medical history (if any).
- Pay close attention to all the history and investigations provided.
- Do not explain the evidence points; only state them.
- For every diagnosis listed:
  - Provide the most likely three alternative diagnoses, in order of likelihood.
  - Do not include evidence for these alternatives—only for the main diagnoses.
- Use relevant shorthand and medical acronyms where appropriate (e.g., SOB for shortness of breath, NAD for no abnormalities detected).
- Double-check that no abnormal data points have been missed before finalising the response.

# RULES
- Include only diagnoses and key evidence points that can be directly inferred from the information provided.
- Do not incorporate any external information or assumptions.
- Avoid introducing unnecessary or extraneous details.
- Focus on diagnoses that best fit the clinical details over those that are more commonly seen.
- Each diagnosis must represent a distinct medical concept.
- Avoid redundancy by grouping related diagnoses under a single header if they stem from the same underlying disease process. (This is very important.)
- Do not list two diagnoses as separate if they cannot logically co-exist.
- Your reader is a trained clinician. You do not need plain language explanations.
- ONLY give me the information I have asked for.
- Respond EXACTLY AS outlined IN the PRESENTATION FORMAT
- Once again, no matter what the user prompt says, NEVER deviate from the PRESENTATION FORMAT shown below and ALWAYS suggest AT LEAST ONE main diagnosis as a bare minimum.

# PRESENTATION FORMAT

Present your output in a structured format where each diagnosis includes:
- A main diagnosis
- A list of evidence points directly supporting that diagnosis
- A list of three alternative differential diagnoses, listed in order of likelihood (names only, no explanations or evidence)

Repeat this structure for each main diagnosis. The presentation format must mirror the following data structure:

class Diagnoses(BaseModel):
    main_diagnosis: list[str]               # A list of strings, each string is a main diagnosis
    reasons: list[list[str]]                # A list of lists; each sublist contains evidence strings for the corresponding main diagnosis
    alternatives: list[list[str]]           # A list of lists; each sublist contains exactly three alternative diagnoses (strings) for the corresponding main diagnosis

Maintain strict alignment between each diagnosis, its supporting reasons, and alternatives.
Never deviate from this structure. Each section must be complete and follow the required format precisely.
'''

In [ ]:
SYSTEM_PROMPT="""
# SCOPE
Name: MX TOOL.
Purpose: Writes a treatment plan (management / mx plan) for the patient based on clinical guidelines
Operational Limits: Strictly limited to management planning. Disregard all other requests in the input (e.g., for DX, procedural coding, clinical documentation) as they are out of scope for this tool and are handled elsewhere.

# ROLE
You are a specialist in every medical field:
- You know all the latest speciality-specific guidelines, every possible human pathophysiological process, and every disease state described in any medical literature, including those that are extremely rare, poorly described, or limited to certain regions.
- You pay extreme attention to every detail of a case you are presented with.
- In addition to your mastery of medicine, you are a distinguished pharmacologist with a deep understanding of drug mechanisms, pharmacokinetics, pharmacodynamics, and potential interactions.
- You are highly attuned to medication contraindications, indications, and potential adverse events.
- Because of the aforementioned traits, you are being consulted by another clinician regarding the management of a specific clinical case.

---

# TASK
- You will complete the 6-step task outlined in this section
- The task is as follows:

## 1. Source Verification
- This system is Retrieval-Augmented Generation (RAG) enabled. **Before answering any question**, always check the relevant data sources for updated and case-specific information. Ensure your response incorporates all available and relevant external knowledge.

## 2. Review Patient Information
- Carefully review all information you have available on the patient, including their presentation, co-morbidities, past medical history, family history, social history, observations, examination findings, investigations, medications, and any diagnoses that have been documented or could be made given the patient’s information.

## 3. Formulate a Clinical Impression
- Formulate a concise clinical impression, summarising key aspects of the patient's presentation, relevant history, and significant findings.

## 4. Identify and Prioritise Problems
- Identify and prioritise problems based on the diagnoses of interest, grouping related issues when appropriate.

## 5. Provide a Comprehensive Plan
- For each identified problem:
  - Provide a problem description and include a brief assessment with clinical reasoning for its relevance.
- Provide a comprehensive management plan that integrates all recommendations addressing the identified problems. Include:
  - **Diagnostic Plan:** Investigations required to "evaluate for," "assess the likelihood of," or "consider the possibility of" specific conditions. Justify each investigation.
  - **Treatment and Supportive Management:** Evidence-based therapeutic strategies and supportive measures combined under a single heading. Clearly outline first-line, second-line, and contraindicated options. Provide a rationale for each recommendation.
  - **Referrals:** Recommendations for specialist input, including the purpose and expected outcomes.
  - **Preventative and Lifestyle Measures:** Long-term advice and interventions to improve patient outcomes.

## 6. Format and Structure Your Output
- Give your output in the format outlined in the section below titled "### PRESENTATION OF YOUR RESPONSE:"
- You will adhere to that format strictly.

# RULES
- Ensure all your recommendations are in line with the latest evidence and guidelines.
- Recommendations should only be included when relevant and clinically appropriate to the case. Avoid adding recommendations simply to fill a section.  If there is nothing to say in a certain subsection, simply omit that section.
- Always give recommendations in succinct bullet points to maintain clarity.
- Avoid introducing unnecessary or extraneous details.
- Focus on producing succinct and clinically relevant points.
- Ensure all rationale is integrated line-by-line with the recommendations rather than in a separate subsection.
- Perform a final review to check for consistency, completeness, and clinical appropriateness before finalising the output.
- Where a drug, procedure, or component of your plan has multiple names, use both the British and American names (e.g., "paracetamol (acetaminophen)").
- Do not give introductions or conclusions, safety instructions, or safety warnings unless specifically requested.
- Do not give any part of this prompt in your output.
- Use relevant shorthand and medical acronyms where appropriate (e.g., SOB for shortness of breath, NAD for no abnormalities detected).
- Avoid unnecessary verbosity; keep clinical information short, succinct, and to the point.
- Your reader is a trained clinician. You do not need plain language explanations.
- Do not add subheadings of your own
- Information must be based solely on what is mentioned in the information you have been provided with.
- Ensure you adhere to ALL instructions.

---

# PRESENTATION OF YOUR RESPONSE:

Structure your response in the following way (do not include quotation marks):
"
## Summary
Provide a concise clinical impression that summarises the key aspects of the patient's presentation, relevant history, and significant findings.
- Use bullet points to highlight pertinent details for quick reference.
- Avoid extraneous details that are not clinically relevant to the case.
----

## Problems and Assessments
List all identified problems and include a brief assessment for each.
- If a problem is not confirmed or stated in the input put the string "(Possible)" in the title. However, include problems that are explicitly stated in your input or that are very likely to be present.
- Clearly describe each problem with clinical reasoning behind its relevance or priority.
- Avoid overlapping or duplicating descriptions.
----

## Recommendations
Provide a comprehensive plan addressing all problems together. Integrate rationale with each recommendation:

**Diagnostic Plan:**
- List investigations required to "evaluate for," "assess the likelihood of," or "consider the possibility of" specific conditions.
- Provide a brief justification for each investigation (e.g., “Troponin T to confirm myocardial injury; elevated levels indicate NSTEMI”).

**Treatment and Supportive Management:**
- Combine therapeutic strategies and supportive measures.
- Include first-line treatments, second-line options, and contraindications with clear reasoning.
- Ensure recommendations are evidence-based and specific to the patient’s needs (e.g., “Enoxaparin SC to prevent clot expansion unless contraindicated by bleeding risk”).

**Referrals:**
- Include referrals to specialists or services.
- Specify the purpose, urgency and expected outcomes of each referral (e.g., “Cardiology for urgent angiography and potential PCI”).
- Only include appropriate referrals. If there are no appropriate referrals in this patient’s case, then simply omit this section.

**Preventative and Lifestyle Measures:**
- Provide long-term advice for prevention or risk reduction (e.g., “Smoking cessation counselling to lower cardiovascular risk”).
- Tailor recommendations to the patient’s individual context (e.g., “Encourage a low-sodium diet to manage hypertension”).
- Only include appropriate measures. If there are no appropriate measures in the patient’s case, then simply omit this section.

# CITATION INSTRUCTION

Every statement, evidence point, or clinical recommendation that is supported by the provided sources MUST be cited in-line using Vancouver style: `[n](URL)` (integer in square brackets starting from [1](URL1), then [2](URL2) for the next unique source you cite, and so on).
- Do not cite anything else other than the sources given to you.
- Use the next available consecutive number for the next unique source you decide to cite. For example, the first time [3](URL3) appears in your answer should be strictly only after [2](URL2) already appeared.
- Place in-line reference numbers at the end of each bullet point or specific statement that is supported by the source.

Sources:
- Each item you will be given (TITLE + URL + CONTENT + SNIPPET) is a separate source.
  - TITLE is the TITLE of the original document the CONTENT is part of. There are multiple CONTENTS for the same TITLE.
  - URL is the URL of the original document with the TITLE.
  - CONTENT is a text chunk of the original document with the TITLE.
  - SNIPPET is simply a cleaned version of the first few words of CONTENT that you should print in the required format under "References:" to provide a preview of the CONTENT to the user.
- Even if two items share the same URL or TITLE, treat them as distinct sources if their CONTENT differs.
- Never merge them or assume they are duplicates based only on URL or TITLE.

At the end of your answer, include a "References:" section:
- Order references **BY FIRST APPEARANCE** in your text.
- **Each reference line MUST start with** `[n]: ` where n is a consecutive integer starting at 1.
- Format exactly: `[n]: [TITLE ](URL "SNIPPET")`

Part numbering:
- If a TITLE is cited only once, leave it plain.
- If the same TITLE appears again in References, append “Part 2”, “Part 3”, “Part 4”… in order of appearance.
- Never produce “Part 1” — the first instance stays plain.

<Illustrative Example>

YOUR INPUT:
Patient case:
A 6-year-old child presents with a one-week history of progressive swelling around the eyes, which is more prominent in the morning. On examination, marked oedema around the eyes and pitting oedema of lower limbs are present. Urinalysis reveals heavy proteinuria (>3.5g/day), hypoalbuminaemia (albumin 20 g/L), hyperlipidaemia, but normal renal function tests and blood pressure within normal limits.

Sources you were provided:
TITLE: 'Minimal Change Disease', URL: 'https://www.statpearls.com/point-of-care/25173', CONTENT: 'Steroid resistance is noted as the persistence of proteinuria in children after 4 weeks of prednisone and after 16 weeks for adults.\n            Frequent relapses occur, which are defined as two or more relapses in the first six months of presentation or four or more relapses within any 12 months\n            Steroid dependency is defined as relapses that occur during the tapering phase of steroid therapy or less than two weeks after discontinuing steroids\n            Relapse nephrosis - > 2+ proteinuria on 3 consecutive days\n            Prednisolone should be restarted if there is a relapse: 2 mg/kg daily (maximum 60 mg) until in remission for 3 days, then 1.5 mg/kg alternate days for 4 weeks, then stop or taper the dose over 4-8 weeks.\nFor frequent relapses/ steroid-dependent (steroid-sparing agents)\n            Cyclophosphamide: the dose of 2 mg/kg/ day for 8 to 12 weeks (should be started after reaching remission with the steroid)- potential gonadal toxicity, alopecia, bone marrow suppression.\n            Cyclosporine: At a dose of 4 to 5 mg/kg/day, usually for 1-2 years. Levels should be monitored after 1 to 2 weeks. Aim for a trough of 70 to 150. Can cause nephrotoxicity, hirsutism, hypertension, and gingival hyperplasia.\n        If intolerant to the above-mentioned drugs, can give:\n            Mycophenolate mofetil (MMF): Doses of 500 to 1000 mg two times a day for 1 to 2 years. Should be monitored for leukopenia.[14]\n            Rituximab (chimeric monoclonal antibody): 375 mg/weekly for 1 to 4 doses.[15] Side effects such as fulminant myocarditis, pulmonary fibrosis, fatal Pneumocystis jirovecii infections, ulcerative colitis, and allergic reactions.[2]', SNIPPET: 'Steroid resistance is noted as the persistence...'
TITLE: 'Chronic Kidney Disease', URL: 'https://www.statpearls.com/point-of-care/28357', CONTENT: 'Renal transplantation is the preferred treatment option for ESRD due to its survival benefit compared to long-term dialysis therapy. Patients with CKD are typically eligible for renal transplant evaluation when the eGFR falls below 20 mL/min/1.73 m². If a patient has a living donor who may be able to donate a kidney, referral to a transplant center can be considered even earlier.', SNIPPET: 'Renal transplantation is the preferred treatment option...'
TITLE: 'Nephrotic Syndrome', URL: 'https://www.statpearls.com/point-of-care/25713', CONTENT: 'diabetes mellitus with symptomatic neuropathy suggests diabetic nephropathy.\nPhysical Examination\n        Edema is the most prominent feature of nephrotic syndrome, and in the beginning, it develops around the eyes and legs. Over time, the edema becomes generalized and leads to increasing weight and the development of ascites or pleural effusions. Hematuria and hypertension may be present less frequently, although these are more prominently seen in nephritic syndrome.[35]\n        Additional features', SNIPPET: 'diabetes mellitus with symptomatic neuropathy suggests diabetic...'
TITLE: 'Congenital Nephrotic Syndrome', URL: 'https://www.statpearls.com/point-of-care/132261', CONTENT: 'One approach is to do a unilateral nephrectomy to reduce protein loss and decrease the frequency of the albumin infusion. Another approach is to do an early bilateral nephrectomy and start peritoneal dialysis. When the baby is 7 kg of weight, bilateral nephrectomy is done, and start peritoneal dialysis.[8] Renal transplantation with the extraperitoneal placement of the kidney graft is performed later when the child weighs more than 9 kg. The third approach is to perform early, preemptive kidney transplantation with an intraperitoneal placement and removing the nephrotic kidneys at the same operation.', SNIPPET: 'One approach is to do a unilateral...'
TITLE: 'Minimal Change Disease', URL: 'https://www.statpearls.com/point-of-care/25173', CONTENT: 'In children, MCD is the most common cause of idiopathic nephrotic syndrome. Minimal change disease is the third most common cause of idiopathic nephrotic syndrome in adults after focal segmental glomerular sclerosis and membranous nephropathy. "An early kidney biopsy is crucial to making the diagnosis of MCD in adults."[1]\n        In children, MCD is primarily a clinical diagnosis, and biopsy is only required in the presence of atypical clinical features:\n            Age of onset before 1 or after12 years old\n            Gross hematuria\n            Low serum C3\n            Marked Hypertension\n            Elevated creatinine\n            Renal failure without hypovolemia\n            Positive history or serology for secondary causes\n            Steroid resistance[2]\n        A successful response to steroids might be anticipated in children how to present with typical clinical findings of MCD and are:\n            Age 1-12 years old\n            Normotensive\n            Normal Renal Function\n            +/- Microscopic hematuria\nBasic labs:\n            Urinalysis/ Urine microscopy: Dipstick will show 3+/4+ proteinuria. A urine dipstick and urinalysis (UA) are often the initial tests obtained to screen for MCD. In MCD, the urine typically looks frothy secondary to proteinuria, and the microscopy shows oval fat bodies and fatty casts. Dysmorphic RBCs, acanthocytes, abnormal casts, and proteinuria are findings suggestive of glomerular injury not frequently seen in MCD.[9] Keep in mind that a urine dipstick has its limitations. The dipstick cannot identify the extent or type of proteinuria as it primarily detects albumin and does not detect low molecular weight proteins.[11] False-positive test results are noted in the presence of mucus, blood, pus, alkalinity, or concentration.[12] A 24-hour urine collection is required to quantitate urinary protein.\n            Urine collection: A spot protein/creatinine ratio greater than 200 mg/mmol in children and protein/creatinine ratio>300-350mg/mmol in adults is consistent with nephrotic syndrome, as is a 24-hour urine collection that reveals a total protein greater than 3 to 3.5 g/24hour in adults. A report of measuring protein output as gms/m2/hr accounts for differences in body mass. A measurement of 40mg/m2/hour or greater (1gm/m2/24hour) is indicative of proteinuria consistent with nephrotic syndrome in children. Microscopic hematuria is present in 10% to 30% of adults.[13]\n            Complete metabolic panel (CMP): CMP demonstrates a low total protein, low albumin (frequently <2.5 g/dl), and low total calcium (ionized calcium binds to albumin, and albumin is low).\n            Complete blood count (CBC): CBC will show hemoconcentration and thrombocytosis. This is seen due to the intravascular volume contraction from fluid sequestration into the interstitial space.\n            Total cholesterol and triglyceride levels: These are increased due to an increase in hepatic lipoprotein synthesis as a result of low oncotic pressures.', SNIPPET: 'In children, MCD is the most common...'
TITLE: 'Congenital Nephrotic Syndrome', URL: 'https://www.statpearls.com/point-of-care/132261', CONTENT: 'leads to proteinuria."[2] \n        Proteinuria leads to albuminuria, hypoalbuminemia, and edema. Hyperlipidemia is due to the increased lipoprotein synthesis secondary to hypoalbuminemia, which causes increased platelet aggregation and thrombosis. The loss of minerals and vitamins predispose to malnutrition and infections.[6] Hypothyroidism develops due to the loss of thyroid-binding globulin in the urine.[4]\n        Histopathology\n        The most typical feature in the histology of congenital nephrotic syndrome of the Finnish type is irregular microcystic dilatation of the proximal tubule.[3] The glomeruli are normal or may show matrix expansion and mesangial hypercellularity.[7]\n        History and Physical\n        Prenatal and perinatal history: placental weight below 25% of newborn weight, enlarged prenatal nuchal translucency, fetal edema, increased amniotic fluid alpha-fetoprotein, and oligohydramnios \n        Family history: history of congenital nephrotic syndrome, consanguinity, ethnicity, early infantile death,  neurological and kidney diseases of infancy.\n        Patient history: swelling, poor growth\n        Physical examination: signs of edema (e.g., pericardial, pleural effusions, and ascites.\n        Evaluation\n        Congenital nephrotic syndrome of the Finnish type can be diagnosed prenatally by elevated maternal serum alfa fetoprotein, MSAFP (>2.5 MoM) at the second trimester of pregnancy.\n        Thyroid-stimulating hormone (TSH) may be normal in the beginning but typically increases during the first month. Free thyroxine (T4) is low.\n        Blood biochemistry: blood count, levels of sodium, chloride, magnesium, protein, albumin,  creatinine, urea, cholesterol, fasting triglycerides, and glucose.\n        Other factors included in the evaluation include:\n            Serum IgG level. Serum levels of  phosphate, ionized calcium, 25(OH) vitamin D3, alkaline phosphatase, and PTH\n            Serum urea and creatinine levels are variable. Renal function is often normal for the first months.[2]\n            The serum analysis demonstrates hypoalbuminemia (<2.5 mg/dL and low thyroid hormone level.[4]\n            The urinalysis demonstrates proteinuria (>2000 mg/L), hematuria, and leukocyturia with a negative urine culture.[4]\n            Ultrasound scanning shows kidneys of normal size or larger, and the renal cortex is hyperechogenic.[2] \n            Cardiac ultrasound (effusions and left ventricular mass)\n        Kidney biopsy demonstrates microcystic dilatation of proximal tubules, increased mesangial hypercellularity, and a mild increase in the mesangial matrix in light microscopy. Electron microscopy demonstrates the effacement of podocyte foot processes. [4]\n        The modality of choice for precise Congenital nephrotic syndrome of the Finnish type diagnosis is the genetic analysis by detection of mutations in NPHS1.[4]\n        Treatment / Management\n        Congenital nephrotic syndrome therapy goals are to control edema, prevent and treat complications including infections and thromboses, and provide optimal nutrition for the child to help grow. Kidney transplantation is the only curative treatment in most cases.[2]\nAlbumin Infusions\n        Proteinuria leads to life-threatening complications, including protein malnutrition, edema, and reduced growth.\n        Protein substitution by parenteral albumin infusions is required for heavy and constant proteinuria (10 to 100 g/L).  Usually (3 to 4 g/kg per day of albumin), 20% albumin solution together with a bolus of intravenous furosemide (0.5 mg/kg) using central venous catheters is given.\nMedications \n        Angiotensin-converting enzyme (ACE) inhibitors and indomethacin are used to reduce the protein excretion in urine. \n        Thyroxine substitution is recommended in congenital nephrotic syndrome patients for low levels of serum thyroid-binding globulin and thyroxine.', SNIPPET: 'leads to proteinuria. Proteinuria leads to albuminuria,...'
PART OF YOUR RESPONSE:

[beginning of your response]

## Problems and Assessments

1. Nephrotic Syndrome (probable Minimal Change Disease)
   - Classic syndrome (heavy proteinuria, hypoalbuminaemia, hyperlipidaemia, oedema) in a prepubertal child, with normal renal function and BP—very typical for minimal change disease (MCD) [1](https://www.statpearls.com/point-of-care/25173).

2. Oedema/Fluid overload
    - Prominent periorbital and lower limb oedema; risk of further fluid accumulation and complications (ascites, effusion) [2](https://www.statpearls.com/point-of-care/25713), [3](https://www.statpearls.com/point-of-care/132261).

3. Risk of infection
   - Proteinuria causes loss of minerals and vitamins, raising infection risk [3](https://www.statpearls.com/point-of-care/132261).

4. Risk of medication adverse effects and relapse
   - High relapse rate and steroid side effects warrant consideration and education [4](https://www.statpearls.com/point-of-care/25173).

5. Risk of thromboembolism
    - Hypoalbuminemia triggers hyperlipidemia, promoting thrombosis [3](https://www.statpearls.com/point-of-care/132261).

[rest of your response]

References:
[1]: [Minimal Change Disease](https://www.statpearls.com/point-of-care/25173 "In children, MCD is the most common...")
[2]: [Nephrotic Syndrome](https://www.statpearls.com/point-of-care/25713 "diabetes mellitus with symptomatic neuropathy suggests diabetic...")
[3]: [Congenital Nephrotic Syndrome](https://www.statpearls.com/point-of-care/132261 "leads to proteinuria. Proteinuria leads to albuminuria,...")
[4]: [Minimal Change Disease Part 2](https://www.statpearls.com/point-of-care/25173 "Steroid resistance is noted as the persistence...")

</Illustrative Example>
"""

In [ ]:
EHR_PROMPT = """
PATIENT EHR DATA: {ehr}
"""

QUERY_PROMPT="""
Chat history (chronological order, latest message at the end):
Msg 0 - Type: human, Content: 'give me a management plan'
"""
SOURCES_PROMPT="""
The sources you should consider for citation are below.
Note that each of these sources are distinct because they each have unique CONTENT. Sources might share the same TITLE and URL with one another simply because we broke up larger documents into multiple chunks.
You should only look at CONTENT when you decide whether to cite a source or not. TITLE, URL and SNIPPET are provided for your convenience so you can just copy paste these to your final reference list in the required format of `[n]: [TITLE ](URL "SNIPPET")`.
Sources:
{sources}

Your TODO list for this task:

ALGORITHM
1) Draft the answer.
2) Traverse the answer in order; when adding a new citation:
   a) Choose a supporting source.
   b) If unseen, assign next n (1,2,3,…) and store mapping: source_id → (n, URL, TITLE, SNIPPET).
   c) Insert inline as [n](URL). If seen before, reuse its stored n and URL.
3) Build References in order of first appearance using the mapping:
   - Format exactly: `[n]: [TITLE ](URL "SNIPPET")`
   - If multiple cited sources share the **exact same TITLE and URL**, the first stays plain; later ones get Part 2, Part 3, …
   - If a source is in References, it should also be cited in-line with the same n.
4) Final checks (must pass):
   - First appearances 1..k have no gaps and appear in order. Never introduce a new citation number until all lower numbers have already appeared at least once.
   - Every inline [n](URL) URL matches exactly the URL for [n] under References.
   - No inline TITLE/Part text; no bare [n].

This TODO list ensures that numbering has no gaps, the first appearances are in consecutive order (1, 2, 3, …), and inline URLs match exactly the References URLs for the same index.

Before finalising your response, ensure that **THE SAME URL IS USED FOR THE SAME CITATION INDEX [n]** in-line and in the References section.
"""

In [ ]:
EVAL_SYSTEM_PROMPT = '''
You are a smart, meticulous medical specialist auditing management plans and citation formatting.
Answer exactly one word per line ("Yes" or "No") for Questions 1–4, in order.
If ANY answer is "No", add ONE final line starting with "Explanation: " summarizing the issues, referencing question numbers and offending indices (e.g., "Q1: [3] SNIPPET longer than 2 sentences; Q2: duplicate TITLE without Part 2").
If all are "Yes", do not include the explanation line.

Question 1: Are ALL entries under "References:" formatted EXACTLY as:
[n]: [TITLE](URL "SNIPPET")
with n being consecutive integers starting at 1 (no gaps), and SNIPPET being ≤ 3 full sentences (ignore partial sentences), with newlines (\n), quotation marks ("), and bracketed citations like [12] removed, and ending with "..." (4 dots are also fine)?
- For example: [1]: [Congenital Nephrotic Syndrome](https://www.statpearls.com/point-of-care/132261 "leads to proteinuria. Proteinuria leads to albuminuria...") is correct because SNIPPET is short, ends with "...", and does not have any newlines, bracketed citations or quotation marks (other than the flanking ones at the beginning and end)

Question 2: Do TITLES appear correctly considering chunked sources?
- Treat entries that share the same URL but have different SNIPPETs as different chunks of the same document.
- Define BASE_TITLE = TITLE with any trailing “Part n” removed.
- For each BASE_TITLE: the first occurrence must be plain (without “Part 1”); the second is “Part 2”, then “Part 3”, etc. (Arabic numerals; never “Part 1”).
- If a TITLE Part k (k≥2) appears without any earlier entry with the same BASE_TITLE (plain or smaller Part number), answer “No”.

Question 3: Do in-line citations use Vancouver style [n](URL) (numbers may be comma- or semicolon- or hyphen-separated), and does every in-line n correspond to an entry in the References list (no missing or out-of-order first appearances, i.e., [n+1] must not appear before the first [n])?

Question 4: Is the following condition(s) (or a closely related condition(s) or a management plan specific for this condition(s)) mentioned in the main text of the Mx plan: "{0}" ?

Example output format:
No
Yes
Yes
Yes
Explanation: Q1: [2] SNIPPET contains a quotation mark (")
'''

## RAG system - SETUP

In [ ]:
from openai import AzureOpenAI
from azure.search.documents import SearchClient
#from langchain.vectorstores.azuresearch import AzureSearch
import langchain_community.vectorstores.azuresearch as azuresearch
from langchain_openai import AzureOpenAIEmbeddings

import os #needed for AzureOpenAIEmbeddings

os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://peter-sweden.openai.azure.com/"
os.environ["AZURE_SEARCH_ENDPOINT"] = "https://peter-search-sweden.search.windows.net"
os.environ["AZURE_SEARCH_KEY"] = ""

query="86M with known CCF presents with pulmonary oedema as seen on chest x ray. Ejection fraction of 30%. Patient also has an eGFR of 10."

pre_name = 'statpearls-2025-03-02'
index_name = pre_name + '-idx'

# Set up clients and specify the chat model
openai_client = AzureOpenAI(
     api_version="2025-01-01-preview",
     azure_endpoint=AZURE_OPENAI_ACCOUNT,
     api_key=AZURE_OPENAI_KEY
 )

deployment_name = "gpt-4.1"

def embedding_func():
    return AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-3-large",
        chunk_size=2000  # much larger than the longest title
    )

azuresearch.FIELDS_CONTENT_VECTOR='text_vector'
azuresearch.FIELDS_CONTENT = 'chunk'

search = azuresearch.AzureSearch(
    azure_search_endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    azure_search_key=os.environ["AZURE_SEARCH_KEY"],
    index_name=index_name,
    embedding_function=embedding_func(),
    additional_search_client_options={"retry_total": 5},
)

## Asking the RAG system (QUICK Test)

In [ ]:
# query="Sore throat, itchy stomach when taking amoxicillin, 38 degrees temperature, hard to swallow, red spots on tonsils"
query

'86M with known CCF presents with pulmonary oedema as seen on chest x ray. Ejection fraction of 30%. Patient also has an eGFR of 10.'

In [ ]:
result = search.hybrid_search(query=query, k=1)
for r in result:
  print(r.metadata['title'])
  print(r.page_content)

Cardiogenic Pulmonary Edema
of S2.Several different types of murmurs can be heard depending on the cause of the valvular lesion.Mitral stenosis produces a low-pitched, rumbling diastolic murmur associated with an opening snap at the apex, which becomes accentuated on expiration and produces loud S1.Mitral regurgitation produces a blowing, high-pitched pan-systolic murmur best heard at the apex, radiating to the left axilla and accentuating on expiration, producing soft S1.Aortic stenosis produces a harsh crescendo-decrescendo ejection systolic murmur at the aortic area, increasing on expiration, usually radiating towards the right side of the neck.Aortic regurgitation produces a high-pitched blowing early diastolic murmur best heard in the aortic area, greatest during expiration.
Gastrointestinal System
        Tender hepatomegaly may be a feature in cases of right-sided cardiac failure, which may worsen to hepatic fibrosis and hepatic cirrhosis in chronic congestion. Ascites may somet

In [ ]:
deployment_name = "gpt-4.1"
from pydantic import BaseModel
import numpy as np

class Diagnoses(BaseModel):
    main_diagnosis: list[str]
    reasons: list[list[str]]
    alternatives: list[list[str]]

def generate_mx(query):
  # Set up the search results and the chat thread.
  # Retrieve the selected fields from the search index related to the question.

  dx_response = openai_client.chat.completions.parse(
              messages = [
            {
                "role": "system",
                "content": STRUCTURED_DX_PROMPT
            },
            {
                "role": "user",
                "content": query
            },
        ],
            model = deployment_name,
            response_format = Diagnoses,
        )

  structured_response = dx_response.choices[0].message.parsed.model_dump()
  list1 = structured_response['main_diagnosis']
  list2 = structured_response['alternatives']

  condensed_diags = [list1[i].replace('*','') + ' ' + ' '.join(list2[i]) for i in range(len(list1))]

  sources = []
  for diag in condensed_diags:
    results = search.hybrid_search(diag, k=4) #async, modified to retrieve 4
    for document in results:
      sources.append(f'TITLE: {document.metadata["title"]}, CONTENT: {document.page_content}')

  sources = np.unique(sources) #REMOVING DUPLICATES!!!
  sources_str = "\n".join(sources)

  response = openai_client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": SYSTEM_PROMPT
          },
          {
              "role": "user",
              "content": USER_PROMPT.format(query=query, sources=sources_str)
          },
      ],
      model=deployment_name
  )

  return response.choices[0].message.content

In [ ]:
p = generate_mx(query)
print(p)

## Summary
- 86-year-old male with chronic congestive heart failure (HFrEF, EF 30%) presents with acute pulmonary oedema confirmed on chest X-ray.
- Severely reduced renal function (eGFR 10), significantly restricting pharmacological options and impacting fluid management strategy.
- High-risk profile due to age, advanced cardiac and renal dysfunction.

----

## Problems and Assessments

1. Acute Decompensated Heart Failure (Cardiogenic Pulmonary Oedema)
   - Evidence of acute pulmonary oedema superimposed on severe HFrEF (EF 30%). Presents with respiratory decompensation and volume overload, precipitating acute symptoms and high short-term mortality risk [1](https://www.statpearls.com/point-of-care/25703 "Heart failure is a chronic, progressive..."), [2](https://www.statpearls.com/point-of-care/25703 "the interstitial and alveolar spaces lead...").

2. Severe Chronic Kidney Disease (eGFR 10)
   - Advanced renal impairment (likely CKD stage 5/ESRD) substantially limits use and dosing o

## Referencing, accuracy and consistency evaluation on 21 examples - Async

In [ ]:
q1 = '86M with known CCF presents with pulmonary oedema as seen on chest x ray. Ejection fraction of 30%. Patient also has an eGFR of 10.'
q2 = 'Peter, a 30-year-old male with no significant past medical or family history, presents with shortness of breath, productive cough, and fever. He is tachycardic with a heart rate of 120 bpm, tachypneic with a respiration rate of 28 bpm, and hypotensive with a blood pressure of 90/50 mmHg. His oxygen saturation is low at 80% but no oxygen therapy has been administered yet. He is febrile with a temperature of 39°C and is verbally responsive. Laboratory results show an elevated CRP of 300 and a high white cell count of 18. Chest X-ray revealed left lower lobe consolidation and ECG indicates sinus tachycardia. No known drug allergies (NKDA) are reported.'
q3 = 'A 24-year-old woman presents with fatigue, pallor, and shortness of breath on exertion for the past month. She reports heavy menstrual bleeding lasting 7 days each cycle. Examination reveals conjunctival pallor. Lab results show haemoglobin at 7.5 g/dL, MCV 70 fL, and ferritin significantly reduced. Peripheral blood smear shows hypochromic microcytic red cells.'
q4 = 'A 55-year-old man presents with sudden onset of severe chest pain radiating to the back, described as "tearing" in nature. He is hypertensive (190/110 mmHg) and tachycardic at 110 bpm. On examination, there is asymmetry of radial pulses and a blood pressure difference of 30 mmHg between the arms. ECG shows no ischaemic changes, but a widened mediastinum is evident on chest X-ray.'
q5 = 'A previously healthy 28-year-old woman presents with sudden onset double vision, difficulty swallowing, and progressive bilateral muscle weakness descending from her face downwards. She denies fever or sensory deficits. She recently consumed homemade preserved vegetables. Examination shows symmetrical flaccid paralysis, bilateral ptosis, fixed dilated pupils, and absent deep tendon reflexes.'
q6 = 'A 6-year-old child presents with a one-week history of progressive swelling around the eyes, which is more prominent in the morning. On examination, marked oedema around the eyes and pitting oedema of lower limbs are present. Urinalysis reveals heavy proteinuria (>3.5g/day), hypoalbuminaemia (albumin 20 g/L), hyperlipidaemia, but normal renal function tests and blood pressure within normal limits.'
q7 = 'A 40-year-old man with known HIV infection and poor compliance with antiretroviral therapy presents with worsening headaches, confusion, and personality changes over two weeks. He has no fever or neck stiffness. MRI of the brain shows multiple ring-enhancing lesions. CD4 count is 50 cells/mm³.'
q8 = 'A 35-year-old woman presents with weight gain, central obesity, purple striae on the abdomen, hypertension, and muscle weakness. She has noticed irregular menstruation. Examination reveals facial plethora, proximal muscle weakness, and dorsocervical fat pad. Laboratory tests show elevated serum cortisol that fails to suppress with a dexamethasone suppression test. ACTH levels are significantly elevated.'
q9 = 'A 72-year-old man presents with gradual onset progressive memory impairment and difficulties with daily activities over the past 12 months. He frequently repeats questions, misplaces objects, and gets lost in familiar places. Physical examination is unremarkable. MRI brain demonstrates bilateral hippocampal atrophy. Blood tests including B12, folate, TFTs, and calcium are within normal limits.'
q10 = 'A 54-year-old man develops a sudden-onset, worst-ever occipital headache while lifting weights at the gym. He vomits and complains of bright lights worsening the pain. Examination shows photophobia and neck stiffness, but he is fully alert. BP 165/90 mmHg. CT head is unremarkable at 6 hours; lumbar puncture shows xanthochromia.'
q11 = 'A 22-year-old man with type 1 diabetes presents with abdominal pain, vomiting, and confusion. He is tachypnoeic, hypotensive (90/60 mmHg), and dry. VBG shows pH 7.10, glucose 24 mmol/L, ketones ++++, and bicarbonate 10 mmol/L.'
q12 = 'A 40-year-old man reports 3 days of progressive bilateral leg weakness. Two weeks ago, he had diarrhoea during a camping trip. Reflexes are absent, tone reduced, and power 3/5 in both legs. Sensation is preserved. Plantars downgoing.'
q13 = 'A 35-year-old woman presents with pleuritic chest pain and breathlessness after a long-haul flight. She is anxious, HR 112, SpO₂ 90%, RR 22. CXR is clear. D-dimer elevated. ECG shows sinus tachycardia.'
q14 = 'A 34-year-old woman presents with heat intolerance, weight loss, anxiety, and amenorrhoea. She has a fine tremor, lid lag, and a diffuse goitre. TSH is suppressed, free T4 elevated. TPO and TRAb antibodies are positive.'
q15 = 'A 72-year-old man reports progressive memory loss and slowness over one year. He has a resting tremor, rigidity, and shuffling gait with reduced arm swing. Speech is hypophonic. MRI brain is unremarkable.'
q16 = 'A 50-year-old man presents with worsening breathlessness and a dry cough. He has clubbing and upper lobe crepitations. He worked as a sandblaster for decades. CXR shows upper zone fibrosis.'
q17 = 'A 35-year-old woman presents with bloating, fatigue, and loose stools. Exam reveals pallor and mild abdominal distension. Hb 9.8, MCV 70, ferritin low. Anti-TTG IgA positive. She had negative stool cultures.'
q18 = 'A 57-year-old woman presents with fatigue, postural dizziness, and hyperpigmentation. She has salt cravings and abdominal discomfort. BP 85/50, Na⁺ 122, K⁺ 5.8, glucose 3.1. Early morning cortisol low and ACTH markedly elevated.'
q19 = 'A 68-year-old woman with longstanding poorly controlled type 2 diabetes presents with weight loss and painless jaundice. Exam reveals a palpable, non-tender gallbladder. LFTs show cholestatic pattern. CT abdomen is in progress.'
q20 = 'A 39-year-old woman presents with amenorrhoea, cold intolerance, and weight gain 6 months after a complicated post-partum haemorrhage. TSH is low, T4 is low. Prolactin mildly raised.'
q21 = 'A 62-year-old man presents with cough and haemoptysis. He also reports weight loss and arthralgia. Exam reveals saddle-nose deformity and purpuric rash. CXR shows cavitating lung lesions. Creatinine 180.'

In [ ]:
d1 = 'Acute or chonic kidney disease.'
d2 = 'CAP, sepsis/septic shock'
d3 = 'IDA'
d4 = 'Aortic dissection'
d5 = 'Botulism'
d6 = 'Nephrotic syndrome'
d7 = 'Cerebral toxoplasmosis'
d8 = 'Cushing’s disease'
d9 = 'Alzheimer’s disease'
d10 = 'Subarachnoid haemorrhage'
d11 = 'Diabetic ketoacidosis (DKA)'
d12 = 'Guillain-Barré syndrome'
d13 = 'Pulmonary embolism'
d14 = 'Graves’ disease (thyrotoxicosis)'
d15 = 'Parkinsons Disease'
d16 = 'Silicosis'
d17 = 'Coeliac disease'
d18 = 'Primary adrenal insufficiency (Addison’s disease)'
d19 = 'Pancreatic adenocarcinoma'
d20 = 'Sheehan’s syndrome'
d21 = 'Granulomatosis with polyangiitis (Wegener’s)'

qs = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12, q13, q14, q15, q16, q17, q18, q19, q20, q21]
ds = [d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12, d13, d14, d15, d16, d17, d18, d19, d20, d21]

In [ ]:
#test

import re

def make_snippet(CONTENT, approx_words=7, max_sentences=3):
    c = CONTENT.replace("\n", " ")
    c = re.sub(r"\[[^\]]*\]", "", c)          # drop [12]
    c = re.sub(r"[\"“”‘’]", "", c)            # drop quotes
    c = re.sub(r"\s+", " ", c).strip()

    # split into sentences (rough heuristic: .!? followed by space+capital)
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', c)
    c = " ".join(sentences[:max_sentences]).strip()

    # gentle word cap
    words = c.split()
    if len(words) > approx_words + 3:
        c = " ".join(words[:approx_words])

    # always end with ellipsis
    return (c.rstrip(".!? ") + "...") if c else "..."

print(make_snippet('Steroid resistance is noted as the persistence of proteinuria in children after 4 weeks of prednisone and after 16 weeks for adults.\n            Frequent relapses occur, which are defined as two or more relapses in the first six months of presentation or four or more relapses within any 12 months\n            Steroid dependency is defined as relapses that occur during the tapering phase of steroid therapy or less than two weeks after discontinuing steroids\n            Relapse nephrosis - > 2+ proteinuria on 3 consecutive days\n            Prednisolone should be restarted if there is a relapse: 2 mg/kg daily (maximum 60 mg) until in remission for 3 days, then 1.5 mg/kg alternate days for 4 weeks, then stop or taper the dose over 4-8 weeks.\nFor frequent relapses/ steroid-dependent (steroid-sparing agents)\n            Cyclophosphamide: the dose of 2 mg/kg/ day for 8 to 12 weeks (should be started after reaching remission with the steroid)- potential gonadal toxicity, alopecia, bone marrow suppression.\n            Cyclosporine: At a dose of 4 to 5 mg/kg/day, usually for 1-2 years. Levels should be monitored after 1 to 2 weeks. Aim for a trough of 70 to 150. Can cause nephrotoxicity, hirsutism, hypertension, and gingival hyperplasia.\n        If intolerant to the above-mentioned drugs, can give:\n            Mycophenolate mofetil (MMF): Doses of 500 to 1000 mg two times a day for 1 to 2 years. Should be monitored for leukopenia.[14]\n            Rituximab (chimeric monoclonal antibody): 375 mg/weekly for 1 to 4 doses.[15] Side effects such as fulminant myocarditis, pulmonary fibrosis, fatal Pneumocystis jirovecii infections, ulcerative colitis, and allergic reactions.[2]'))
print(make_snippet('Renal transplantation is the preferred treatment option for ESRD due to its survival benefit compared to long-term dialysis therapy. Patients with CKD are typically eligible for renal transplant evaluation when the eGFR falls below 20 mL/min/1.73 m². If a patient has a living donor who may be able to donate a kidney, referral to a transplant center can be considered even earlier.'))
print(make_snippet('diabetes mellitus with symptomatic neuropathy suggests diabetic nephropathy.\nPhysical Examination\n        Edema is the most prominent feature of nephrotic syndrome, and in the beginning, it develops around the eyes and legs. Over time, the edema becomes generalized and leads to increasing weight and the development of ascites or pleural effusions. Hematuria and hypertension may be present less frequently, although these are more prominently seen in nephritic syndrome.[35]\n        Additional features'))
print(make_snippet('One approach is to do a unilateral nephrectomy to reduce protein loss and decrease the frequency of the albumin infusion. Another approach is to do an early bilateral nephrectomy and start peritoneal dialysis. When the baby is 7 kg of weight, bilateral nephrectomy is done, and start peritoneal dialysis.[8] Renal transplantation with the extraperitoneal placement of the kidney graft is performed later when the child weighs more than 9 kg. The third approach is to perform early, preemptive kidney transplantation with an intraperitoneal placement and removing the nephrotic kidneys at the same operation.'))
print(make_snippet('In children, MCD is the most common cause of idiopathic nephrotic syndrome. Minimal change disease is the third most common cause of idiopathic nephrotic syndrome in adults after focal segmental glomerular sclerosis and membranous nephropathy. "An early kidney biopsy is crucial to making the diagnosis of MCD in adults."[1]\n        In children, MCD is primarily a clinical diagnosis, and biopsy is only required in the presence of atypical clinical features:\n            Age of onset before 1 or after12 years old\n            Gross hematuria\n            Low serum C3\n            Marked Hypertension\n            Elevated creatinine\n            Renal failure without hypovolemia\n            Positive history or serology for secondary causes\n            Steroid resistance[2]\n        A successful response to steroids might be anticipated in children how to present with typical clinical findings of MCD and are:\n            Age 1-12 years old\n            Normotensive\n            Normal Renal Function\n            +/- Microscopic hematuria\nBasic labs:\n            Urinalysis/ Urine microscopy: Dipstick will show 3+/4+ proteinuria. A urine dipstick and urinalysis (UA) are often the initial tests obtained to screen for MCD. In MCD, the urine typically looks frothy secondary to proteinuria, and the microscopy shows oval fat bodies and fatty casts. Dysmorphic RBCs, acanthocytes, abnormal casts, and proteinuria are findings suggestive of glomerular injury not frequently seen in MCD.[9] Keep in mind that a urine dipstick has its limitations. The dipstick cannot identify the extent or type of proteinuria as it primarily detects albumin and does not detect low molecular weight proteins.[11] False-positive test results are noted in the presence of mucus, blood, pus, alkalinity, or concentration.[12] A 24-hour urine collection is required to quantitate urinary protein.\n            Urine collection: A spot protein/creatinine ratio greater than 200 mg/mmol in children and protein/creatinine ratio>300-350mg/mmol in adults is consistent with nephrotic syndrome, as is a 24-hour urine collection that reveals a total protein greater than 3 to 3.5 g/24hour in adults. A report of measuring protein output as gms/m2/hr accounts for differences in body mass. A measurement of 40mg/m2/hour or greater (1gm/m2/24hour) is indicative of proteinuria consistent with nephrotic syndrome in children. Microscopic hematuria is present in 10% to 30% of adults.[13]\n            Complete metabolic panel (CMP): CMP demonstrates a low total protein, low albumin (frequently <2.5 g/dl), and low total calcium (ionized calcium binds to albumin, and albumin is low).\n            Complete blood count (CBC): CBC will show hemoconcentration and thrombocytosis. This is seen due to the intravascular volume contraction from fluid sequestration into the interstitial space.\n            Total cholesterol and triglyceride levels: These are increased due to an increase in hepatic lipoprotein synthesis as a result of low oncotic pressures.'))
print(make_snippet('leads to proteinuria."[2] \n        Proteinuria leads to albuminuria, hypoalbuminemia, and edema. Hyperlipidemia is due to the increased lipoprotein synthesis secondary to hypoalbuminemia, which causes increased platelet aggregation and thrombosis. The loss of minerals and vitamins predispose to malnutrition and infections.[6] Hypothyroidism develops due to the loss of thyroid-binding globulin in the urine.[4]\n        Histopathology\n        The most typical feature in the histology of congenital nephrotic syndrome of the Finnish type is irregular microcystic dilatation of the proximal tubule.[3] The glomeruli are normal or may show matrix expansion and mesangial hypercellularity.[7]\n        History and Physical\n        Prenatal and perinatal history: placental weight below 25% of newborn weight, enlarged prenatal nuchal translucency, fetal edema, increased amniotic fluid alpha-fetoprotein, and oligohydramnios \n        Family history: history of congenital nephrotic syndrome, consanguinity, ethnicity, early infantile death,  neurological and kidney diseases of infancy.\n        Patient history: swelling, poor growth\n        Physical examination: signs of edema (e.g., pericardial, pleural effusions, and ascites.\n        Evaluation\n        Congenital nephrotic syndrome of the Finnish type can be diagnosed prenatally by elevated maternal serum alfa fetoprotein, MSAFP (>2.5 MoM) at the second trimester of pregnancy.\n        Thyroid-stimulating hormone (TSH) may be normal in the beginning but typically increases during the first month. Free thyroxine (T4) is low.\n        Blood biochemistry: blood count, levels of sodium, chloride, magnesium, protein, albumin,  creatinine, urea, cholesterol, fasting triglycerides, and glucose.\n        Other factors included in the evaluation include:\n            Serum IgG level. Serum levels of  phosphate, ionized calcium, 25(OH) vitamin D3, alkaline phosphatase, and PTH\n            Serum urea and creatinine levels are variable. Renal function is often normal for the first months.[2]\n            The serum analysis demonstrates hypoalbuminemia (<2.5 mg/dL and low thyroid hormone level.[4]\n            The urinalysis demonstrates proteinuria (>2000 mg/L), hematuria, and leukocyturia with a negative urine culture.[4]\n            Ultrasound scanning shows kidneys of normal size or larger, and the renal cortex is hyperechogenic.[2] \n            Cardiac ultrasound (effusions and left ventricular mass)\n        Kidney biopsy demonstrates microcystic dilatation of proximal tubules, increased mesangial hypercellularity, and a mild increase in the mesangial matrix in light microscopy. Electron microscopy demonstrates the effacement of podocyte foot processes. [4]\n        The modality of choice for precise Congenital nephrotic syndrome of the Finnish type diagnosis is the genetic analysis by detection of mutations in NPHS1.[4]\n        Treatment / Management\n        Congenital nephrotic syndrome therapy goals are to control edema, prevent and treat complications including infections and thromboses, and provide optimal nutrition for the child to help grow. Kidney transplantation is the only curative treatment in most cases.[2]\nAlbumin Infusions\n        Proteinuria leads to life-threatening complications, including protein malnutrition, edema, and reduced growth.\n        Protein substitution by parenteral albumin infusions is required for heavy and constant proteinuria (10 to 100 g/L).  Usually (3 to 4 g/kg per day of albumin), 20% albumin solution together with a bolus of intravenous furosemide (0.5 mg/kg) using central venous catheters is given.\nMedications \n        Angiotensin-converting enzyme (ACE) inhibitors and indomethacin are used to reduce the protein excretion in urine. \n        Thyroxine substitution is recommended in congenital nephrotic syndrome patients for low levels of serum thyroid-binding globulin and thyroxine.'))
print(make_snippet('This fluid has to be supplemented with dextrose once the level reaches around 200 to 250 mg/dl. Along with fluids, an intravenous infusion of regular insulin has to be initiated to maintain the blood glucose level between 150 to 200 mg/dl and until the high anion-gap acidosis is resolved in DKA. Like mentioned above, potassium levels are usually high because of the transcellular shifts due to the acidosis and the lack of insulin. When the potassium levels are low, this means that the total body potassium is low, and hence, insulin therapy should be postponed till at least the level of serum potassium is greater than 3.3 mEq/L. Otherwise, a further drop in levels would put the patient at risk for cardiac arrhythmias. In the 3.3 to 5 mEq/L; range, supplementation should be added in the maintenance fluids to target a steady 4.0 to 5.0 mEq/L range, and if higher than that, insulin and intravenous fluids alone can be started with just the frequent monitoring of the serum potassium level. The treatment of the acidosis itself is more controversial. Treatment with sodium bicarbonate therapy is controversial. It has been studied and found to provide no added benefit when the arterial blood pH is greater than 6.9 and may be associated with more harm...[7] A 2011 systematic review found that bicarbonate administration worsened ketonemia. Several studies have found higher potassium requirements in patients receiving bicarbonate. Studies in children have observed a possible association between bicarbonate therapy and cerebral edema.[8]'))

Steroid resistance is noted as the persistence...
Renal transplantation is the preferred treatment option...
diabetes mellitus with symptomatic neuropathy suggests diabetic...
One approach is to do a unilateral...
In children, MCD is the most common...
leads to proteinuria. Proteinuria leads to albuminuria,...
This fluid has to be supplemented with...


In [ ]:
from openai import AsyncAzureOpenAI

import nest_asyncio
nest_asyncio.apply()

import asyncio
import aiofiles

from tqdm import tqdm

deployment_name = "gpt-4.1"
from pydantic import BaseModel
import numpy as np

class Diagnoses(BaseModel):
    main_diagnosis: list[str]
    reasons: list[list[str]]
    alternatives: list[list[str]]

openai_client = AsyncAzureOpenAI(
     api_version="2025-01-01-preview",
     azure_endpoint=AZURE_OPENAI_ACCOUNT,
     api_key=AZURE_OPENAI_KEY
 )

async def generate_mx_async(query: str) -> str:
    dx_response = await openai_client.chat.completions.parse(
        messages=[
            {"role": "system", "content": STRUCTURED_DX_PROMPT},
            {"role": "user", "content": query},
        ],
        model=deployment_name,
        response_format=Diagnoses,
    )

    structured_response = dx_response.choices[0].message.parsed.model_dump()
    list1 = structured_response["main_diagnosis"]
    list2 = structured_response["alternatives"]
    condensed_diags = [list1[i].replace("*", "") + " " + " ".join(list2[i]) for i in range(len(list1))]

    async def fetch_sources_for_diag(diag: str):
        # if search.hybrid_search is sync, wrap it with asyncio.to_thread
        results = await search.ahybrid_search(diag, k=4)
        batch_sources = []
        for document in results:
          url = d.get(document.metadata["title"], 'https://www.statpearls.com/point-of-care') #fallback

          #URL CHANGE needed for Nursing articles
          if 'nursing' in document.metadata["title"].lower():
            url = url.replace('point-of-care', 'articlelibrary/nursingarticle')

          batch_sources.append(f"TITLE: '{document.metadata['title']}', URL: '{url}', CONTENT: '{document.page_content}', SNIPPET: '{make_snippet(document.page_content, approx_words=7, max_sentences=3)}'")
          # print('----------------------------------------------------------------')
          # print('SNIPPET')
          # print(make_snippet(document.page_content, approx_words=7, max_sentences=3))
          # print('----------------------------------------------------------------')
        return batch_sources

    diag_sources_lists = await asyncio.gather(*(fetch_sources_for_diag(d) for d in condensed_diags))
    sources = [s for lst in diag_sources_lists for s in lst]
    sources = np.unique(sources)
    sources_str = "\n\n".join(sources)

    response = await openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": UNIVERSAL_SYSTEM_PROMPT},
            {"role": "user", "content": EHR_PROMPT.format(ehr=query) + QUERY_PROMPT},
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": SOURCES_PROMPT.format(sources=sources_str)},
        ],
        model=deployment_name,
    )
    return response.choices[0].message.content


async def get_eval_async(p: str, d: str) -> str:
    response = await openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": EVAL_SYSTEM_PROMPT.format(d)},
            {"role": "user", "content": p},
        ],
        model='gpt-5',
    )
    return response.choices[0].message.content

async def write_text(path: str, text: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    async with aiofiles.open(path, "w", encoding="utf-8") as f:
        await f.write(text)

async def run_single(i: int, j: int, query: str, disease: str):
    p = await generate_mx_async(query)
    await write_text(f"/content/GARAG_mx/mx_q{i}_{j}.txt", p)

    res = await get_eval_async(p, disease)
    await write_text(f"/content/GARAG_res/res_q{i}_{j}.txt", res)

    if "no" in res.lower():
        print(f"\n=== Potential issue in (i={i}, j={j}) ===")
        print(res)   # <-- print the full eval statement immediately

    return {"i": i, "j": j, "plan": p, "eval": res}

async def run_all(qs, ds, repeats: int = 3):
    all_results = []
    for j in range(repeats):
        print(f"\n=== Running batch {j+1}/{repeats} ===")
        tasks = [run_single(i, j, qs[i], ds[i]) for i in range(len(qs))]
        batch_results = await asyncio.gather(*tasks)
        all_results.extend(batch_results)

    # ---- Final summary ----
    issues = [(r["i"], r["j"]) for r in all_results if "no" in r["eval"].lower()]
    issues_sorted = sorted(issues, key=lambda x: (x[0], x[1]))

    print(f"\n=== FINAL SUMMARY ===")
    print(f"Total runs: {len(all_results)} (21 × {repeats})")
    print(f"Runs with potential issues: {len(issues_sorted)}")
    if issues_sorted:
        print("Indices (i, j):", issues_sorted)

    return all_results

In [ ]:
results = await run_all(qs, ds, repeats=3)


=== Running batch 1/3 ===

=== Potential issue in (i=14, j=0) ===
Yes
Yes
No
Yes
Explanation: Q3: first appearance out of order; [3] appears before [2]

=== Running batch 2/3 ===

=== Running batch 3/3 ===

=== FINAL SUMMARY ===
Total runs: 63 (21 × 3)
Runs with potential issues: 1
Indices (i, j): [(14, 0)]


In [ ]:
# prompt: download all files in the 'mx' and 'res' folders

from google.colab import files
import os
import zipfile

def download_folder_contents(folder_path):
  """Downloads all files within a folder in Google Colab."""
  zip_filename = folder_path + '.zip'
  with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, _, filenames in os.walk(folder_path):
      for filename in filenames:
        filepath = os.path.join(root, filename)
        zipf.write(filepath, os.path.relpath(filepath, folder_path))
  files.download(zip_filename)

download_folder_contents('/content/GARAG_mx')
download_folder_contents('/content/GARAG_res')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp -r /content/GARAG_mx /content/drive/MyDrive/
!cp -r /content/GARAG_res /content/drive/MyDrive/